In [ ]:
!mkdir Dreambooth
!cd Dreambooth
!mkdir input_path output_path class_path  

Install the required libraries

In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py

In [ ]:
%pip install -q gradio

Huggingface login

Load the input images in /home/Dreambooth/input_path folder

In [ ]:
#create a concepts_list with their paths

concepts_list = [
    {
        "instance_prompt":      "photo of kak person",#kak is the keyword/token of our images
        "class_prompt":         "photo of a person",
        "instance_data_dir":    "/home/Dreambooth/input_path",
        "class_data_dir":       "/home/Dreambooth/class_path"
    }
]


In [ ]:

import json

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

In [ ]:
# Model training

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-base" \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir="/home/Dreambooth/output_path" \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=1500 \
  --save_interval=500 \
  --save_sample_prompt="photo of kak person" \
  --concepts_list="concepts_list.json"

In [ ]:
WEIGHTS_DIR="/home/Dreambooth/output_path/1500"

Creating Pipeline

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

In [ ]:
try:
    pipe
except NameError:
    pipe = StableDiffusionPipeline.from_pretrained(
        WEIGHTS_DIR,
        torch_dtype=torch.float16,
    ).to("cuda")


In [ ]:
pipe

Inference

In [ ]:
import gradio as gr

In [ ]:
def inference(prompt, num_samples):
    all_images = [] 
    images = pipe(prompt, num_images_per_prompt=num_samples, num_inference_steps=50, guidance_scale=7.5).images
    all_images.extend(images)
    return all_images

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="prompt")
            samples = gr.Slider(label="Samples",value=1)
            run = gr.Button(value="Run")
        with gr.Column():
            gallery = gr.Gallery(show_label=False)

    run.click(inference, inputs=[prompt,samples], outputs=gallery)
    gr.Examples([["a photo of vish person as captain america", 1,1]], [prompt,samples], gallery, inference, cache_examples=False)


demo.launch(server_name="0.0.0.0",server_port=6006)